In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import warnings
import os

warnings.filterwarnings("ignore")

In [ ]:
# ========== CONFIGURATION ==========
DATA_PATH = "../data/engineered_features.csv"
TARGET = "retail_price_£_per_kWh"
MLFLOW_TRACKING_URI = "http://13.221.17.34:8000/"
EXPERIMENT_NAME = "UK Energy - Feature Selection"
TEST_SIZE = 0.2
RANDOM_STATE = 42
OPTUNA_TRIALS = 50

In [ ]:
print("UK ENERGY PRICE PREDICTION - FEATURE SELECTION")

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"\nMLflow Tracking URI: {MLFLOW_TRACKING_URI}")
print(f"Experiment: {EXPERIMENT_NAME}")

UK ENERGY PRICE PREDICTION - FEATURE SELECTION


2025/10/28 12:37:21 INFO mlflow.tracking.fluent: Experiment with name 'UK Energy - Feature Selection' does not exist. Creating a new experiment.



MLflow Tracking URI: http://13.221.17.34:8000/
Experiment: UK Energy - Feature Selection


In [ ]:
print("\Loading data...")
df = pd.read_csv(DATA_PATH)
print(f"   Loaded data shape: {df.shape}")
print(f"   Date range: {df['datetime'].min()} to {df['datetime'].max()}")

\Loading data...
   Loaded data shape: (2059, 104)
   Date range: 2025-08-02 05:00:00+00:00 to 2025-10-26 23:00:00+00:00


In [ ]:
# Exclude non-feature columns
excluded_cols = [TARGET, "datetime"]

# Select only scaled features + temporal features for modeling
features = [
    col
    for col in df.columns
    if col.startswith("scaled_")
    or col
    in [
        "hour_sin",
        "hour_cos",
        "month_sin",
        "month_cos",
        "is_weekend",
        "is_peak_hour",
        "is_night",
        "hour",
        "day_of_week",
        "month",
    ]
]

# Remove rows with NaN (from lag/rolling features)
df_clean = df[features + [TARGET]].dropna()

X = df_clean[features]
y = df_clean[TARGET]

print(f"   Total features: {len(features)}")
print(f"   Clean samples: {len(X)}")
print(f"   Target: {TARGET}")

   Total features: 50
   Clean samples: 2059
   Target: retail_price_£_per_kWh


In [ ]:
print("\nSplitting data...")
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    shuffle=False,  # Time-series: no shuffle
)
print(f"   Train: {len(X_train)} samples")
print(f"   Test: {len(X_test)} samples")


Splitting data...
   Train: 1647 samples
   Test: 412 samples


In [ ]:
def calculate_metrics(y_true, y_pred):
    """Calculate regression metrics."""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "mape": mape}

In [ ]:
def log_model_to_mlflow(model, model_name, params, metrics, feature_importances):
    """Log model, params, metrics, and artifacts to MLflow."""
    with mlflow.start_run(run_name=model_name):
        # Log parameters
        mlflow.log_params(params)

        # Log metrics
        mlflow.log_metrics(metrics)

        # Log model
        if "XGBoost" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

        # Log feature importances as artifact
        importance_df = pd.DataFrame(
            {
                "feature": feature_importances.index,
                "importance": feature_importances.values,
            }
        )
        importance_path = f"artifacts/{model_name}_feature_importance.csv"
        os.makedirs("artifacts", exist_ok=True)
        importance_df.to_csv(importance_path, index=False)
        mlflow.log_artifact(importance_path)

        print(
            f"   ✅ Logged {model_name} - RMSE: {metrics['rmse']:.4f}, R²: {metrics['r2']:.4f}"
        )

In [ ]:
def plot_feature_importance(importances, title, filename):
    """Plot and save feature importance."""
    plt.figure(figsize=(12, 8))
    sns.barplot(x=importances.values, y=importances.index, palette="viridis")
    plt.title(title, fontsize=16, fontweight="bold")
    plt.xlabel("Importance Score", fontsize=12)
    plt.ylabel("Feature", fontsize=12)
    plt.tight_layout()

    os.makedirs("plots", exist_ok=True)
    plot_path = f"plots/{filename}"
    plt.savefig(plot_path, dpi=300, bbox_inches="tight")
    plt.close()

    return plot_path

In [ ]:
# ========== RANDOM FOREST - OPTUNA ==========
print("\Training Random Forest with Optuna...")


def rf_objective(trial):
    """Optuna objective for Random Forest."""
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 5, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical(
            "max_features", ["sqrt", "log2", 0.5]
        ),
        "random_state": RANDOM_STATE,
    }

    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)

    return mse

\Training Random Forest with Optuna...


In [ ]:
rf_study = optuna.create_study(direction="minimize", study_name="RF_Optimization")
rf_study.optimize(rf_objective, n_trials=OPTUNA_TRIALS, show_progress_bar=True)

print("\n   🏆 Best RF Parameters:")
for key, value in rf_study.best_params.items():
    print(f"      {key}: {value}")
print(f"   📉 Best MSE: {rf_study.best_value:.6f}")

[I 2025-10-28 12:38:05,048] A new study created in memory with name: RF_Optimization


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-28 12:38:07,838] Trial 0 finished with value: 0.0038133357607976043 and parameters: {'n_estimators': 480, 'max_depth': 26, 'min_samples_split': 19, 'min_samples_leaf': 7, 'max_features': 'log2'}. Best is trial 0 with value: 0.0038133357607976043.
[I 2025-10-28 12:38:09,933] Trial 1 finished with value: 0.0034065789398443373 and parameters: {'n_estimators': 297, 'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.0034065789398443373.
[I 2025-10-28 12:38:12,023] Trial 2 finished with value: 0.003655726176274721 and parameters: {'n_estimators': 446, 'max_depth': 5, 'min_samples_split': 15, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.0034065789398443373.
[I 2025-10-28 12:38:13,430] Trial 3 finished with value: 0.003404328265297105 and parameters: {'n_estimators': 187, 'max_depth': 13, 'min_samples_split': 12, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 3 with va

In [ ]:
# Train best RF model
best_rf = RandomForestRegressor(**rf_study.best_params, random_state=RANDOM_STATE)
best_rf.fit(X_train, y_train)
rf_preds = best_rf.predict(X_test)
rf_metrics = calculate_metrics(y_test, rf_preds)

In [ ]:
# Get feature importances
rf_importances = pd.Series(
    best_rf.feature_importances_, index=X_train.columns
).sort_values(ascending=False)

In [ ]:
# Log to MLflow
log_model_to_mlflow(
    best_rf,
    "RandomForest_Optuna",
    rf_study.best_params,
    rf_metrics,
    rf_importances[:25],
)

2025/10/28 12:42:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 12:42:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


   ✅ Logged RandomForest_Optuna - RMSE: 0.0528, R²: 0.7069
🏃 View run RandomForest_Optuna at: http://13.221.17.34:8000/#/experiments/1/runs/936f74e4eedc4ab38033cb33c40aa1c0
🧪 View experiment at: http://13.221.17.34:8000/#/experiments/1


In [ ]:
# Plot top 25 features
rf_plot_path = plot_feature_importance(
    rf_importances[:25],
    "Random Forest - Top 25 Feature Importances",
    "rf_feature_importance.png",
)

In [ ]:
with mlflow.start_run(run_name="RandomForest_Optuna"):
    mlflow.log_artifact(rf_plot_path)

print("\n   📊 Random Forest Metrics:")
print(f"      RMSE: {rf_metrics['rmse']:.4f}")
print(f"      MAE: {rf_metrics['mae']:.4f}")
print(f"      R²: {rf_metrics['r2']:.4f}")
print(f"      MAPE: {rf_metrics['mape']:.2f}%")

🏃 View run RandomForest_Optuna at: http://13.221.17.34:8000/#/experiments/1/runs/ae55d9df7e8f4017b4f2d030b8a47f57
🧪 View experiment at: http://13.221.17.34:8000/#/experiments/1

   📊 Random Forest Metrics:
      RMSE: 0.0528
      MAE: 0.0317
      R²: 0.7069
      MAPE: inf%


In [ ]:
# ========== XGBOOST - OPTUNA ==========
print("Training XGBoost with Optuna...")


def xgb_objective(trial):
    """Optuna objective for XGBoost."""
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 2.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 2.0),
        "random_state": RANDOM_STATE,
        "tree_method": "hist",
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)

    return mse

Training XGBoost with Optuna...


In [ ]:
xgb_study = optuna.create_study(direction="minimize", study_name="XGB_Optimization")
xgb_study.optimize(xgb_objective, n_trials=OPTUNA_TRIALS, show_progress_bar=True)

print("\n   🏆 Best XGBoost Parameters:")
for key, value in xgb_study.best_params.items():
    print(f"      {key}: {value}")
print(f"   📉 Best MSE: {xgb_study.best_value:.6f}")

[I 2025-10-28 12:43:25,049] A new study created in memory with name: XGB_Optimization


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-28 12:43:25,626] Trial 0 finished with value: 0.004981590168387297 and parameters: {'n_estimators': 362, 'max_depth': 15, 'learning_rate': 0.2670941788368, 'subsample': 0.8992315849382913, 'colsample_bytree': 0.7746349751334731, 'gamma': 0.7732654855637093, 'reg_alpha': 1.5129340640161706, 'reg_lambda': 1.8945117425612599}. Best is trial 0 with value: 0.004981590168387297.
[I 2025-10-28 12:43:25,864] Trial 1 finished with value: 0.004382110795073445 and parameters: {'n_estimators': 171, 'max_depth': 14, 'learning_rate': 0.0888424181492604, 'subsample': 0.8757596559273678, 'colsample_bytree': 0.7600312534735136, 'gamma': 0.7488999246881732, 'reg_alpha': 0.4394327948755372, 'reg_lambda': 0.14145196381803804}. Best is trial 1 with value: 0.004382110795073445.
[I 2025-10-28 12:43:26,110] Trial 2 finished with value: 0.00516610944848832 and parameters: {'n_estimators': 135, 'max_depth': 10, 'learning_rate': 0.13354178110239368, 'subsample': 0.6035784999753094, 'colsample_bytree':

In [ ]:
best_xgb = XGBRegressor(
    **xgb_study.best_params, random_state=RANDOM_STATE, tree_method="hist"
)
best_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
xgb_preds = best_xgb.predict(X_test)
xgb_metrics = calculate_metrics(y_test, xgb_preds)

In [ ]:
xgb_importances = pd.Series(
    best_xgb.feature_importances_, index=X_train.columns
).sort_values(ascending=False)

In [ ]:
# Log to MLflow
log_model_to_mlflow(
    best_xgb, "XGBoost_Optuna", xgb_study.best_params, xgb_metrics, xgb_importances[:25]
)

2025/10/28 12:44:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 12:44:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


   ✅ Logged XGBoost_Optuna - RMSE: 0.0552, R²: 0.6789
🏃 View run XGBoost_Optuna at: http://13.221.17.34:8000/#/experiments/1/runs/ad008a6ed4c74b40b467311cac984c5b
🧪 View experiment at: http://13.221.17.34:8000/#/experiments/1


In [ ]:
# Plot top 25 features
xgb_plot_path = plot_feature_importance(
    xgb_importances[:25],
    "XGBoost - Top 25 Feature Importances",
    "xgb_feature_importance.png",
)

In [ ]:
with mlflow.start_run(run_name="XGBoost_Optuna"):
    mlflow.log_artifact(xgb_plot_path)

print("\n   📊 XGBoost Metrics:")
print(f"      RMSE: {xgb_metrics['rmse']:.4f}")
print(f"      MAE: {xgb_metrics['mae']:.4f}")
print(f"      R²: {xgb_metrics['r2']:.4f}")
print(f"      MAPE: {xgb_metrics['mape']:.2f}%")

🏃 View run XGBoost_Optuna at: http://13.221.17.34:8000/#/experiments/1/runs/46f315c3533249b9a771c9d6792626c9
🧪 View experiment at: http://13.221.17.34:8000/#/experiments/1

   📊 XGBoost Metrics:
      RMSE: 0.0552
      MAE: 0.0328
      R²: 0.6789
      MAPE: inf%


In [ ]:
# ========== COMPARISON TABLE ==========
print("\n" + "=" * 60)
print("📊 TOP 25 FEATURES BY MODEL")
print("=" * 60)

comparison_df = pd.DataFrame(
    {
        "Feature": rf_importances[:25].index,
        "RF_Importance": rf_importances[:25].values,
        "XGB_Importance": xgb_importances[rf_importances[:25].index].values,
        "Avg_Importance": (
            rf_importances[:25].values
            + xgb_importances[rf_importances[:25].index].values
        )
        / 2,
    }
).sort_values("Avg_Importance", ascending=False)

print("\n" + comparison_df.to_string(index=False))


📊 TOP 25 FEATURES BY MODEL

                         Feature  RF_Importance  XGB_Importance  Avg_Importance
             scaled_price_lag_1h       0.451231        0.247177        0.349204
            scaled_price_lag_24h       0.220397        0.209974        0.215185
            scaled_renewable_pct       0.103522        0.109703        0.106613
                        hour_sin       0.053399        0.050366        0.051882
                            hour       0.026983        0.042721        0.034852
         scaled_uk_gen_biomass_%       0.027005        0.024069        0.025537
         scaled_carbon_per_price       0.030752        0.012993        0.021872
  scaled_carbon_intensity_actual       0.023332        0.013894        0.018613
            scaled_carbon_lag_1h       0.001711        0.028679        0.015195
                        hour_cos       0.014302        0.015590        0.014946
             scaled_wind_lag_24h       0.000877        0.025561        0.013219
           

In [ ]:
# Save comparison table
comparison_path = "artifacts/feature_importance_comparison.csv"
comparison_df.to_csv(comparison_path, index=False)
print(f"\n💾 Saved comparison table to: {comparison_path}")


💾 Saved comparison table to: artifacts/feature_importance_comparison.csv


In [ ]:
print("\n6️⃣ Creating combined feature importance plot...")

fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# RF plot
sns.barplot(
    ax=axes[0],
    x=rf_importances[:25].values,
    y=rf_importances[:25].index,
    palette="viridis",
)
axes[0].set_title("Random Forest - Top 25 Features", fontsize=16, fontweight="bold")
axes[0].set_xlabel("Importance Score", fontsize=12)
axes[0].set_ylabel("Feature", fontsize=12)

# XGBoost plot
sns.barplot(
    ax=axes[1],
    x=xgb_importances[:25].values,
    y=xgb_importances[:25].index,
    palette="plasma",
)
axes[1].set_title("XGBoost - Top 25 Features", fontsize=16, fontweight="bold")
axes[1].set_xlabel("Importance Score", fontsize=12)
axes[1].set_ylabel("")

plt.tight_layout()
combined_plot_path = "plots/combined_feature_importance.png"
plt.savefig(combined_plot_path, dpi=300, bbox_inches="tight")
plt.close()


6️⃣ Creating combined feature importance plot...


In [ ]:
# ========== MODEL COMPARISON ==========
print("📈 MODEL PERFORMANCE COMPARISON")

comparison_metrics = pd.DataFrame(
    {
        "Model": ["Random Forest", "XGBoost"],
        "RMSE": [rf_metrics["rmse"], xgb_metrics["rmse"]],
        "MAE": [rf_metrics["mae"], xgb_metrics["mae"]],
        "R²": [rf_metrics["r2"], xgb_metrics["r2"]],
        "MAPE (%)": [rf_metrics["mape"], xgb_metrics["mape"]],
    }
)

print("\n" + comparison_metrics.to_string(index=False))

# Determine best model
best_model_name = (
    "Random Forest" if rf_metrics["rmse"] < xgb_metrics["rmse"] else "XGBoost"
)
print(f"\n🏆 Best Model: {best_model_name}")

📈 MODEL PERFORMANCE COMPARISON

        Model     RMSE      MAE       R²  MAPE (%)
Random Forest 0.052768 0.031691 0.706851       inf
      XGBoost 0.055230 0.032807 0.678864       inf

🏆 Best Model: Random Forest


In [ ]:
# ========== SAVE RESULTS ==========
import json

print("\Saving results...")

results = {
    "experiment": EXPERIMENT_NAME,
    "mlflow_uri": MLFLOW_TRACKING_URI,
    "data_path": DATA_PATH,
    "target": TARGET,
    "n_features": len(features),
    "n_samples": len(X),
    "train_size": len(X_train),
    "test_size": len(X_test),
    "optuna_trials": OPTUNA_TRIALS,
    "best_model": best_model_name,
    "rf_metrics": rf_metrics,
    "xgb_metrics": xgb_metrics,
    "rf_best_params": rf_study.best_params,
    "xgb_best_params": xgb_study.best_params,
}


results_path = "artifacts/feature_selection_results.json"
with open(results_path, "w") as f:
    # Convert numpy types to native Python types for JSON serialization
    json_results = {
        k: (float(v) if isinstance(v, (np.floating, np.integer)) else v)
        for k, v in results.items()
        if k not in ["rf_metrics", "xgb_metrics", "rf_best_params", "xgb_best_params"]
    }
    json_results["rf_metrics"] = {k: float(v) for k, v in rf_metrics.items()}
    json_results["xgb_metrics"] = {k: float(v) for k, v in xgb_metrics.items()}
    json_results["rf_best_params"] = rf_study.best_params
    json_results["xgb_best_params"] = xgb_study.best_params
    json.dump(json_results, f, indent=2)

print(f"   ✅ Saved results to: {results_path}")

\Saving results...
   ✅ Saved results to: artifacts/feature_selection_results.json
